In [1]:
import pandas as pd
import numpy as np
import torch
import tensorflow as tf
import nltk
import pickle
nltk.download('punkt')
from tqdm import tqdm

#from nltk.tokenize import word_tokenize
#from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import time
from datetime import timedelta

2024-12-12 12:06:19.520752: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-12 12:06:19.538298: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-12 12:06:19.543710: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-12 12:06:19.557194: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-12 12:06:20.596421: W tensorflow/compiler/tf2

In [2]:
print(np.__version__)
print(torch.__version__)

2.0.1
1.13.0+cu117


In [3]:
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

if cuda_available:
    print(f"CUDA version: {torch.version.cuda}")

num_gpus = torch.cuda.device_count()
print(f"Number of GPUs: {num_gpus}")

if num_gpus > 0:
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

CUDA available: True
CUDA version: 11.7
Number of GPUs: 1
GPU 0: NVIDIA A100-SXM4-80GB


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
from torchtext.vocab import GloVe
glove = GloVe(name='42B', dim=300) # 42B tokens, 1.9M vocab, uncased, 300d vectors, 1.75 GB download -> English

In [6]:
with open('/home/basilmusyaffa19/Skripsi Basil/Embedding Matrix/Dataset Inggris/english_keras_tokenizer.pickle', 'rb') as handle:
    keras_tokenizer = pickle.load(handle)
    
vocab_size = len(keras_tokenizer.word_index) + 1
print('Vocab size:', vocab_size)

Vocab size: 6588


In [ ]:
df = pd.read_excel('/home/basilmusyaffa19/Skripsi Basil/Dataset/FIX/clean_UCI_22112024.xlsx', engine='openpyxl')
df

,label,teks
0,1,go jurong point crazy available bugis great wo...
1,1,ok lar joking wif oni
2,1,free entry wkly comp win fa cup final tkts may...
3,1,dun say early hor already say
4,1,nah think go usf life around though
...,...,...
4994,1,time tried contact pound prize claim easy call...
4995,1,going esplanade fr home
4996,1,pity mood suggestion
4997,1,guy bitching acted like interested buying some...


In [ ]:
print("Jumlah data sebelum:", len(df))

# Menghapus data kosong
df = df.replace('', np.nan).dropna()
# Hapus NaN
df = df.dropna(subset=['teks'])
# Menghapus nilai float
df = df[~df['teks'].apply(lambda x: isinstance(x, float))]
# Menghapus semua baris yang duplikat
df = df.drop_duplicates(subset=['teks'], keep='first')

print("Jumlah data setelah:", len(df))

Jumlah data sebelum: 4999
Jumlah data setelah: 4998


In [ ]:
def check_coverage(vocab_list, glove):
    embedding_words = set(glove.itos)
    num_words_found = 0
    oov = []

    for word in tqdm(vocab_list):
        if word in embedding_words:
            num_words_found += 1
        else:
            oov.append(word)

    total_words = len(vocab_list)
    print('Found embeddings for {:.2%} of vocab'.format(num_words_found / total_words))
    print('Number of words not found: {}'.format(len(oov)))
    
    return oov

In [10]:
vocab_list = list(keras_tokenizer.word_index.keys())
oov = check_coverage(vocab_list, glove)

print("\nBeberapa kata yang tidak ditemukan:")
for word in oov[:10]:
    print(f"'{word}'")
    
df_oov = pd.DataFrame(oov, columns=['Kata OOV'])
df_oov.to_excel('/home/basilmusyaffa19/Skripsi Basil/Embedding Matrix/kata_oov_inggris.xlsx', index=False)

100%|██████████| 6587/6587 [00:00<00:00, 506524.65it/s]

Found embeddings for 91.15% of vocab
Number of words not found: 583

Beberapa kata yang tidak ditemukan:
'freemsg'
'savamob'
'bslvyl'
'yijue'
'custcare'
'themob'
'netcollex'
'txtauction'
'linerental'
'urgnt'


In [11]:
embedding_dim = 300
embedding_matrix = np.zeros((vocab_size, embedding_dim))
embedding_matrix.shape

(6588, 300)

In [12]:
def initialize_embedding_matrix(tokenizer, oov_words, embedding_matrix, model, embedding_dim=300):
    start_time = time.time()
    word_embeddings = {}
    
    for word, i in tqdm(tokenizer.word_index.items(), desc="Creating embedding matrix"): 
        if word in oov_words:
            random_vector = np.random.uniform(-0.25, 0.25, embedding_dim)
            random_vector = random_vector / np.linalg.norm(random_vector) * np.sqrt(embedding_dim)
            embedding_matrix[i] = random_vector
            word_embeddings[i] = (word, random_vector)
        else:  
            embedding_vector = model[word].numpy()
            embedding_matrix[i] = embedding_vector
            word_embeddings[i] = (word, embedding_vector)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print(f"\nTotal waktu pembuatan embedding matrix: {elapsed_time}s")
    
    return embedding_matrix, word_embeddings

In [13]:
embedding_matrix_glove, word_embeddings_glove = initialize_embedding_matrix(
    keras_tokenizer,
    oov,
    embedding_matrix,
    glove
)

Creating embedding matrix: 100%|██████████| 6587/6587 [00:00<00:00, 44485.04it/s]


Total waktu pembuatan embedding matrix: 0.15183734893798828s


In [14]:
embedding_matrix_glove

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.10418   ,  0.1406    ,  0.23841999, ...,  0.36063999,
        -0.12285   , -0.30285999],
       [-0.1961    ,  0.071137  , -0.12630001, ..., -0.28173   ,
         0.22459   , -0.023248  ],
       ...,
       [-0.13383999,  0.52635002, -0.22437   , ..., -0.31380001,
         0.34816   , -0.13925   ],
       [-0.36144   , -0.25823   ,  0.043368  , ..., -0.22351   ,
        -0.19698   , -0.016459  ],
       [ 0.094189  ,  0.41624001, -0.51201999, ...,  0.13727   ,
         0.46888   , -0.16497   ]])

In [15]:
for idx, (word, vector) in list(word_embeddings_glove.items())[:5]:
    print(f"Index: {idx}")
    print(f"Word: {word}")
    vector_np = np.array(vector) if not isinstance(vector, np.ndarray) else vector
    print(f"Vector (first 10 values): {vector_np[:10]}")
    print("---")

Index: 1
Word: call
Vector (first 10 values): [-0.10418  0.1406   0.23842  0.28409 -0.23904  0.62945 -3.3813   0.59411
  0.05484 -0.337  ]
---
Index: 2
Word: get
Vector (first 10 values): [-0.1961     0.071137  -0.1263    -0.26382    0.19078    0.27746
 -4.2662     0.0061844  0.093023  -0.65369  ]
---
Index: 3
Word: ur
Vector (first 10 values): [-0.13217  0.68169 -0.33498  0.16939 -0.29244 -0.16021 -2.8784   0.19715
  0.54313 -0.44206]
---
Index: 4
Word: go
Vector (first 10 values): [ 0.094418  0.26803  -0.18872  -0.34682   0.17336   0.5337   -3.8678
 -0.2175    0.33643  -0.63875 ]
---
Index: 5
Word: ok
Vector (first 10 values): [ 0.05973  0.11751 -0.19544 -0.2859   0.34065  0.11716 -2.8535   0.39046
  0.69778 -0.67266]
---


In [16]:
target_word = "freemsg"

word_found = False
for idx, (word, vector) in word_embeddings_glove.items():
    if word == target_word:
        print(f"Index: {idx}")
        print(f"Word: {word}")
        print(f"Vector: {vector}")
        word_found = True
        break

if not word_found:
    print(f"Kata '{target_word}' tidak ditemukan")

Index: 515
Word: freemsg
Vector: [-0.1717406   0.22183211 -1.66577952  1.53995452  0.76546314 -1.64845336
 -1.28138181  0.35215167  1.40190268 -1.24780031  0.6106988  -1.2729654
  0.64123784  1.12478378 -1.57432934  0.89725768  0.95133113 -1.65202324
  0.33633001  0.44771978  0.38977887  0.17062223 -0.6844741  -1.57921496
 -1.63487087 -0.93545945 -1.02690043  0.30111914  1.17773265 -0.55030938
  1.58569097  1.03298917  0.24081842 -1.31572299 -0.94720028 -0.15650836
 -1.12460942 -1.51911945  1.12558968 -0.36682209 -1.38956415 -0.71162591
 -1.65690883 -0.10879413  0.62160919  0.43722576  0.48907955  1.30795345
  1.65518228  0.63733328  1.04760398  0.5513509   1.02392627  1.1982408
 -0.13092216  0.60965289 -1.51979402  0.75108821 -1.28106535 -0.19340008
 -0.03832161  1.28427106 -0.88047771 -0.29737741  0.64159057  0.08196647
 -0.32955561  0.28709161 -0.34721292 -0.38175929 -0.20388313 -1.10168868
 -0.2337588   0.92517062  1.07066229  1.47457061 -1.67632487 -1.55214275
 -0.40847418  0.8878

In [17]:
path = '/home/basilmusyaffa19/Skripsi Basil/Embedding Matrix/Dataset Inggris/Hasil Embedding/21 Nov/embedding_matrix_glove42B_300D_21112024.npy'
np.save(path, embedding_matrix_glove)

In [18]:
import os
file_size = os.path.getsize('/home/basilmusyaffa19/Skripsi Basil/Embedding Matrix/Dataset Inggris/Hasil Embedding/21 Nov/embedding_matrix_glove42B_300D_21112024.npy')
print(f"Ukuran file: {file_size/1024/1024:.2f} MB")

Ukuran file: 15.08 MB
